# Train a Convolutional Neural Network (CNN) on EEG data

In [11]:
from bundle.DataCraft import * 
import numpy as np

data = load_sentence_eeg_prob_data()

if data:
    # Print example of converted item
    print(f"Loaded {len(data)} items.")
    
    # Convert to NumPy array to inspect the full shape
    eeg_array = np.array(data[0]["eeg_chunk"])
    print("Shape of EEG chunk:", eeg_array.shape)

    if data:
        print("Example of first item:", data[1]["character"])
        print("Example of first item:", data[1]["eeg_chunk"][0])

Attempting to load processed data from: ../../data/sentences_eeg.pkl
Successfully loaded processed data.
Loaded 5 items.
Shape of EEG chunk: (31, 78, 64)
Example of first item: H
Example of first item: [[ 0.7758755   0.9237902   0.79516953 ...  1.7251105   1.8952785
   1.7748973 ]
 [ 0.6073626   0.8665243   0.700175   ...  1.5955708   1.8149357
   1.6690725 ]
 [ 0.47421762  0.85209244  0.63459575 ...  1.3974723   1.6194837
   1.4821903 ]
 ...
 [-0.75434214 -1.0152866  -0.5195334  ... -0.3559552  -0.29637173
  -0.70139366]
 [-0.59730214 -0.8856115  -0.3951398  ... -0.4825735  -0.5561047
  -0.848363  ]
 [-0.5027137  -0.74316084 -0.2996873  ... -0.64054483 -0.853964
  -1.004107  ]]
